# Helmholtz 1D - Mock Cycle 2-level Predictions

## Goal
For the 1D Helhmholtz operator $A = -\Delta + k^2 I$, compare the quality of SVD coarsening and pointwise coarsening by the mock cycle asymptotic convergence factor. "pt" donotes pointwise coarsening.

## Discretization
* Fixed periodic domain with $n=16$ points and different $kh$ values..
* 3-point finite difference $A^h = [1, -2 + (kh)^2, 1]$. 
* Kaczmarz relaxation. For comparison, for $kh=0$ only we also look at Gauss-Seidel in lexicographic ordering.

In [142]:
import logging
import numpy as np
import helmholtz as hm
import pandas as pd
import matplotlib.pyplot as plt
import sklearn.metrics.pairwise
import sys
from numpy.ma.testutils import assert_array_almost_equal
from numpy.linalg import eig, norm

%load_ext autoreload
%autoreload 2

np.set_printoptions(linewidth=500, precision=2, suppress=False)
for handler in logging.root.handlers[:]: logging.root.removeHandler(handler)
logging.basicConfig(stream=sys.stdout, level=logging.WARN, format="%(levelname)-8s %(message)s",
                    datefmt="%a, %d %b %Y %H:%M:%S")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Mock Cycle Convergence Factor

In [143]:
def create_svd_coarsening(level):
    # Generate relaxed test matrix.
    n = level.a.shape[0]
    x = hm.run.random_test_matrix((n,))
    lam = 0
    b = np.zeros_like(x)
    x, _, _ = hm.run.run_iterative_method(level.operator, lambda x, lam: (level.relax(x, b, lam), lam),
                                          x, lam, num_sweeps=100)
    # Generate coarse variables (R) based on a window of x.
    aggregate_size = 4
    x_aggregate_t = x[:aggregate_size].transpose()
    r, s = hm.coarsening.create_coarsening(x_aggregate_t, 0.1)
    #print(s, r.asarray().shape)
    #print(r.asarray())

    # Convert to sparse matrix + tile over domain.
    r_csr = r.tile(n // aggregate_size)
    return r_csr

def create_pointwise_coarsening(level):
    aggregate_size = 2
    r = hm.coarsening.Coarsener(np.array([[1, 0]]))
    # Convert to sparse matrix + tile over domain.
    domain_size = level.a.shape[0]
    r_csr = r.tile(domain_size // aggregate_size)
    return r_csr

def mock_cycle_conv_factor(n, kh, relax, coarsening, num_relax_sweeps):
    a = hm.linalg.helmholtz_1d_operator(kh, n)
    level = hm.multilevel.Level.create_finest_level(a)

    if relax == "kaczmarz":
        relaxer = lambda x, b: level.relax(x, b, lam=0)
    elif relax == "gs":
        gs_relaxer = hm.relax.GsRelaxer(a)
        relaxer = lambda x, b: gs_relaxer.step(x, b)
    else:
        raise Exception("Unsupported relaxation type {}".format(relax))

    if coarsening == "svd":
        r = create_svd_coarsening(level)
    elif coarsening == "pt":
        r = create_pointwise_coarsening(level)
#        print(r.todense())
    else:
        raise Exception("Unsupported coarsening type {}".format(coarsening))

    mock_cycle = hm.mock_cycle.MockCycle(relaxer, r, num_relax_sweeps)
    x = hm.run.random_test_matrix((n,), num_examples=1)
    lam = 0
    x, _, conv_factor = hm.run.run_iterative_method(
        level.operator, lambda x, lam: (mock_cycle(x), lam), x, lam, num_sweeps=10)
    return conv_factor

In [151]:
kh_values = np.linspace(0, 1, num=11)
nu_values = range(1, 5)
coarsening_values = ("svd", "pt")
n = 16

def conv_factor(n, coarsening):
    gs = np.array([[mock_cycle_conv_factor(n, 0, "gs", coarsening, nu) for nu in nu_values]])
    conv_factor = np.array([[mock_cycle_conv_factor(n, kh, "kaczmarz", coarsening, nu) 
                             for nu in nu_values] for kh in kh_values])
    return np.concatenate((gs, conv_factor))

def conv_factor_table(n):
    result = np.concatenate(
        tuple(conv_factor(n, coarsening) for coarsening in coarsening_values),
        axis=1)
    return pd.DataFrame(result, 
                          index=("0, GS", ) + 
                          tuple(("{:.1f}".format(kh) for kh in kh_values)),
                         columns=tuple("{}, nu={}".format(coarsening, nu) for coarsening in coarsening_values
                                      for nu in nu_values))

### n = 16

In [145]:
conv_factor_table(16)

,"svd, nu=1","svd, nu=2","svd, nu=3","svd, nu=4","pt, nu=1","pt, nu=2","pt, nu=3","pt, nu=4"
"kh=0, GS",0.300533,0.105639,0.056037,0.019441,0.654017,0.630762,0.587276,0.574472
kh = 0.0,0.445054,0.213577,0.141337,0.118209,0.491654,0.572888,0.467348,0.470840
kh = 0.1,0.396339,0.212484,0.133937,0.123708,0.481602,0.544721,0.491252,0.494086
kh = 0.2,0.397110,0.234945,0.137160,0.125500,0.472625,0.538052,0.513050,0.481763
kh = 0.3,0.453923,0.243931,0.133330,0.122465,0.441007,0.539197,0.490087,0.470185
kh = 0.4,0.433647,0.234932,0.132582,0.125585,0.528445,0.571313,0.479878,0.487357
kh = 0.5,0.424864,0.234511,0.135248,0.139725,0.532542,0.534068,0.443925,0.475002
kh = 0.6,0.283452,0.174645,0.076170,0.048881,0.572861,0.533371,0.478811,0.472896
kh = 0.7,0.529431,0.239821,0.103330,0.133537,0.516041,0.495408,0.509730,0.494437
kh = 0.8,0.565695,0.251684,0.115148,0.110239,0.608919,0.454623,0.492968,0.506061


### n = 32

In [152]:
result = conv_factor_table(32)
result

,"svd, nu=1","svd, nu=2","svd, nu=3","svd, nu=4","pt, nu=1","pt, nu=2","pt, nu=3","pt, nu=4"
"kh=0, GS",0.301883,0.101180,0.072631,0.019443,0.653081,0.596022,0.575826,0.564301
0.0,0.449331,0.215508,0.150941,0.115500,0.472116,0.560310,0.489087,0.489645
0.1,0.413000,0.217741,0.158459,0.114801,0.494809,0.583084,0.481841,0.501458
0.2,0.439179,0.227993,0.148739,0.126050,0.509497,0.580854,0.519091,0.497073
0.3,0.466347,0.232146,0.155594,0.135283,0.516379,0.548195,0.480077,0.485840
0.4,0.491309,0.251937,0.164532,0.135457,0.536154,0.545695,0.525667,0.487568
0.5,0.475090,0.248484,0.145818,0.136765,0.540303,0.552768,0.539201,0.476876
0.6,0.504729,0.280108,0.156279,0.135473,0.575331,0.545929,0.540570,0.469962
0.7,0.601126,0.263020,0.158911,0.126414,0.558712,0.504360,0.522231,0.466345
0.8,0.561854,0.300202,0.158359,0.111313,0.617257,0.450026,0.502674,0.473962


## Conclusions
* SVD coarsening exhibits better convergence rates than pointwise. The rate improves with $nu$, while for pointwise it is bounded by $0.5$.
* Gauss-Seidel is a better smoother for the Poisson case (as it well known).
* Almost the same efficiency is maintained for all values of $0 \leq kh \leq 1$.
* The result is independent of $n$.

### Question
For $kh = 0$ and Gauss-Seidel, the predicted 2-level convergence factor does not improve with $\nu$ and does not get below $\sim 0.5$, even though in practice we know that it does and that pointwise coarsening is a perfectly good coarsening of the Laplace operator.
* Why are we not getting a quantitative prediction with the mock cycle? (We could compare with local mode analysis to gain an insight into the slowest-to-converge component.)
* Can we trust it for other $kh$ values?

In [153]:
print(result.to_latex(float_format=lambda x: '%.2f' % x))

\begin{tabular}{lrrrrrrrr}
\toprule
{} &  svd, nu=1 &  svd, nu=2 &  svd, nu=3 &  svd, nu=4 &  pt, nu=1 &  pt, nu=2 &  pt, nu=3 &  pt, nu=4 \\
\midrule
kh=0, GS &       0.30 &       0.10 &       0.07 &       0.02 &      0.65 &      0.60 &      0.58 &      0.56 \\
0.0      &       0.45 &       0.22 &       0.15 &       0.12 &      0.47 &      0.56 &      0.49 &      0.49 \\
0.1      &       0.41 &       0.22 &       0.16 &       0.11 &      0.49 &      0.58 &      0.48 &      0.50 \\
0.2      &       0.44 &       0.23 &       0.15 &       0.13 &      0.51 &      0.58 &      0.52 &      0.50 \\
0.3      &       0.47 &       0.23 &       0.16 &       0.14 &      0.52 &      0.55 &      0.48 &      0.49 \\
0.4      &       0.49 &       0.25 &       0.16 &       0.14 &      0.54 &      0.55 &      0.53 &      0.49 \\
0.5      &       0.48 &       0.25 &       0.15 &       0.14 &      0.54 &      0.55 &      0.54 &      0.48 \\
0.6      &       0.50 &       0.28 &       0.16 &       0.14 &   